In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "E:\anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "E:\anaconda\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "E:\anaconda\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "E:\anaconda\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "E:\anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "E:\anaconda\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "E:\anaconda\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "E:\anaconda\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "E:\anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "E:\anaconda\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "E:\anaconda\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "E:\anaconda\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "E:\anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "E:\anaconda\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "E:\anaconda\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "E:\anaconda\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

AttributeError: _ARRAY_API not found

In [2]:
seed = 356437451

In [3]:
train = pd.read_csv('data_train.csv')
X_ini = train.iloc[:,2:]
y_ini = train.iloc[:,1]

In [4]:
sp = RepeatedStratifiedKFold(random_state=seed,n_repeats=3,n_splits=5)

In [5]:
import hyperopt
from hyperopt import hp

In [6]:
def objective(param):
    aucs = []
    for train_index,test_index in sp.split(X_ini,y_ini):
        X_train = X_ini.iloc[train_index,:]
        X_vali = X_ini.iloc[test_index,:]
        y_train = y_ini[train_index]
        y_vali = y_ini[test_index]
        model = RandomForestClassifier(random_state=seed,
                                       n_estimators=param['n_estimators'],
                                       max_depth=param['max_depth'],
                                       min_samples_split=param['min_samples_split'],
                                       min_samples_leaf=param['min_samples_leaf'])
        model.fit(X_train,y_train)
        pro_vali = model.predict_proba(X_vali)[:,1]
        auc_vali = roc_auc_score(y_vali,pro_vali)
        aucs.append(auc_vali)
    return -np.mean(aucs)

In [7]:
#超参数搜索范围，根据数据集不同进行修改
space = {
    'n_estimators':hp.choice('n_estimators',range(2,50)),
    'max_depth':hp.choice('max_depth',range(1,3)),
    'min_samples_split':hp.choice('min_samples_split',range(2,50)),
    'min_samples_leaf':hp.choice('min_samples_leaf',range(2,50)),
}

In [8]:
best_param = hyperopt.fmin(objective,space,hyperopt.tpe.suggest,max_evals=100)

100%|█████████████████████████████████████████████| 100/100 [00:51<00:00,  1.93trial/s, best loss: -0.9917616334283001]


In [9]:
best_param

{'max_depth': np.int64(0),
 'min_samples_leaf': np.int64(22),
 'min_samples_split': np.int64(25),
 'n_estimators': np.int64(28)}

In [10]:
#这里注意range()范围与上面的搜索空间保持一致
model = RandomForestClassifier(random_state=seed,
                               n_estimators=range(2,50)[best_param['n_estimators']],
                               max_depth=range(1,3)[best_param['max_depth']],
                               min_samples_split=range(2,50)[best_param['min_samples_split']],
                               min_samples_leaf=range(2,50)[best_param['min_samples_leaf']])
model.fit(X_ini,y_ini)

RandomForestClassifier(max_depth=1, min_samples_leaf=24, min_samples_split=27,
                       n_estimators=30, random_state=356437451)

In [11]:
pro_train = model.predict_proba(X_ini)[:,1]
print('训练集AUC={:.3f}'.format(roc_auc_score(y_ini,pro_train)))

训练集AUC=0.993


In [12]:
df_train = pd.DataFrame({
    'ID':train['ID'],
    'True':y_ini,
    'Pre':pro_train
})
df_train.to_csv('RF_train.csv',index=False)

In [13]:
# Load and predict for multiple test datasets
test_files = ['data_test1.csv', 'data_test2.csv', 'data_test3.csv', 
                  'data_test4.csv', 'data_test5.csv', 'data_test6.csv', 'data_test7.csv', 
                  'data_test8.csv', 'data_test9.csv']
for test_file in test_files:
    test = pd.read_csv(test_file)
    X_test = test.iloc[:,2:]
    y_test = test.iloc[:,1]

    pro_test = model.predict_proba(X_test)[:,1]

    print(f'{test_file} 测试集AUC={roc_auc_score(y_test, pro_test):.3f}')

    # Save the predictions for each test dataset
    df_test = pd.DataFrame({
        'ID': test['ID'],
        'True': y_test,
        'Pre': pro_test
    })
    df_test.to_csv(f'RF_{test_file}_predictions.csv', index=False)

data_test1.csv 测试集AUC=0.891
data_test2.csv 测试集AUC=0.722
data_test3.csv 测试集AUC=0.781
data_test4.csv 测试集AUC=0.707
data_test5.csv 测试集AUC=0.883
data_test6.csv 测试集AUC=0.764
data_test7.csv 测试集AUC=0.990
data_test8.csv 测试集AUC=0.967
data_test9.csv 测试集AUC=0.812


In [14]:
import joblib
#save model
joblib.dump(model, 'saved_model/RF.pkl')

['saved_model/RF.pkl']